In [2]:
import matplotlib as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

# added
from folder import KFolder
import numpy as np
import os
from PIL import Image
from Augmentation import ToAugmantate

In [3]:
def loss_plot(hist):
    train_epochs = list(range(1, len(hist.history['loss']) + 1))
    valid_epochs = list(range(1, len(hist.history['val_loss']) + 1))
    train_losses = hist.history['loss']
    valid_losses = hist.history['val_loss']

    train_min_loss = min(dict(zip(train_epochs, train_losses)).items(), key=lambda x: x[1])
    valid_min_loss = min(dict(zip(valid_epochs, valid_losses)).items(), key=lambda x: x[1])

    plt.plot(train_epochs, hist.history['loss'], linewidth=1, color='blue', label='Training Loss')
    plt.plot(train_min_loss[0], train_min_loss[1], color='black', marker='o',
             label=f'Training Best Epoch - {train_min_loss[0]}')
    plt.plot(valid_epochs, hist.history['val_loss'], linewidth=1, color='red', label='Validation Loss')
    plt.plot(valid_min_loss[0], valid_min_loss[1], color='black', marker='o',
             label=f'Validation Best Epoch - {valid_min_loss[0]}')
    plt.legend(loc='best')
    plt.title(label='Training & Validation Loss')
    plt.xlabel(xlabel='Epochs')
    plt.ylabel(ylabel='Loss')
    plt.show()

In [4]:
def accuracy_plot(hist):
  epochs = list(range(1, len(hist.history['accuracy']) + 1))

  plt.plot(epochs, hist.history['accuracy'], marker='o', linewidth=1, label='Training Accuracy', color='blue')
  plt.plot(epochs, hist.history['val_accuracy'], marker='o', linewidth=1, label='Validation Accuracy', color='red')
  plt.title(label='Training & Validation Accuracy')
  plt.legend(loc='best')
  plt.xlabel(xlabel='Epochs')
  plt.ylabel(ylabel='Accuracy')
  plt.show()

In [5]:
def get_model():
    model = keras.models.Sequential(layers=[
        Conv2D(filters=32, kernel_size=(3, 3), input_shape=(150, 150, 3),
            padding='same', activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dropout(rate=0.2),
        Dense(units=256, activation='relu'),
        Dense(units=4, activation='softmax')
    ])
    return model

DON'T RUN EVERY TIME, ONLY IF U WANT TO SHUFFLE TRAIN <-> TESTS

In [6]:
# import shutil
# output_dir = 'splitted_dataset'
# if os.path.exists(output_dir):
#    shutil.rmtree(output_dir, ignore_errors=True)

# os.makedirs(output_dir)
# import splitfolders
# splitfolders.ratio('dataset', output=output_dir, seed=1337, ratio=(.9, 0.1))

In [7]:
def load_images(path: str, class_id: int) -> list:
    content_list = os.listdir(path)
    files_list = [os.path.join(path, content_thing) for content_thing in content_list if os.path.isfile(os.path.join(path, content_thing))]
    imgs_list = []
    labels_list = []
    for file_path in files_list:
        temp_img = Image.open(file_path)
        numpy_image = np.asarray(temp_img.resize((150,150)))
        imgs_list.append(numpy_image)
        labels_list.append(class_id)
    return np.array(imgs_list), np.array(labels_list)

# load train images
main_path = "splitted_dataset/"
cataract = main_path+"train/cataract"
diabetic_retinopathy = main_path+"train/diabetic_retinopathy"
glaucoma = main_path+"train/glaucoma"
normal = main_path+"train/normal"

In [8]:
images_cataract, labels_cataract = load_images(cataract, 1)
images_diabetic_retinopathy, labels_retinopathy = load_images(diabetic_retinopathy, 2)
images_glaucoma, labels_glaucoma = load_images(glaucoma, 3)
images_normal, labels_normal = load_images(normal, 0)

In [9]:
# merge tables
x = np.concatenate([images_cataract,images_diabetic_retinopathy,images_glaucoma,images_normal])
y = np.concatenate([labels_cataract,labels_retinopathy,labels_glaucoma,labels_normal])
del images_cataract, images_diabetic_retinopathy, images_glaucoma, images_normal, labels_cataract,labels_retinopathy,labels_glaucoma,labels_normal

In [10]:
images_cataract, labels_cataract = load_images(cataract, 1)
images_diabetic_retinopathy, labels_retinopathy = load_images(main_path+"val/diabetic_retinopathy", 2)
images_glaucoma, labels_glaucoma = load_images(main_path+"val/glaucoma", 3)
images_normal, labels_normal = load_images(main_path+"val/normal", 0)

In [11]:
X_test = np.concatenate([images_cataract,images_diabetic_retinopathy,images_glaucoma,images_normal])
Y_test = np.concatenate([labels_cataract,labels_retinopathy,labels_glaucoma,labels_normal])
del images_cataract, images_diabetic_retinopathy, images_glaucoma, images_normal, labels_cataract,labels_retinopathy,labels_glaucoma,labels_normal

In [12]:
train_data_gen = ToAugmantate()
test_data_gen = ToAugmantate(False)

In [13]:
kfolder = KFolder(3, x, y)

Computing folds


100%|██████████| 3/3 [00:01<00:00,  1.59it/s]


In [14]:
# import tensorflow as tf
# from dataset import get_data
# from model import get_model
# import os
# import numpy as np
# import matplotlib.pyplot as plt
# import segmentation_models as sm
# from folder import KFolder
for i, fold in enumerate(kfolder.folds):
    callbacks = [tf.keras.callbacks.ModelCheckpoint(f"./best_model_fold_{i}.h5", save_best_only=True, verbose=1)]
    callbacks = [tf.keras.callbacks.ModelCheckpoint("./best_model.h5", save_best_only=True, verbose=1)]
    model = get_model()
    model.compile(loss='mean_squared_logarithmic_error', optimizer='adam')

    X_train, y_train, X_valid, y_valid = fold

    model.fit(x=X_train, y=y_train, validation_data=(X_valid, y_valid), batch_size=2, epochs=100, callbacks=callbacks, verbose=1)

    del model
    best_model = tf.keras.models.load_model(f"./best_model_fold_{i}.h5")

    evaluation = best_model.evaluate(X_test, Y_test)

    with open(f"./results_fold_{i}.txt",'w') as f:
        f.write(f"test loss: {evaluation[0]}\n")
        f.write(f"accuracy: {evaluation[1]}\n")
        f.write(f"dice-score: {evaluation[2]}\n")
    print(100*'--')


2023-06-11 09:06:26.774192: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-11 09:06:26.955298: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-11 09:06:26.955357: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-11 09:06:26.962814: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-11 09:06:26.962878: I tensorflow/compile

Epoch 1/100


2023-06-11 09:06:52.358672: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600


: 

: 